## Apache Beam Pipelines
- サンプルパイプラインを実行して，Beamの使い方を確認する

In [ ]:
# !pip install google-cloud-core==1.6.0 apache_beam[gcp]==2.28.0

In [1]:
import re

import apache_beam as beam
from apache_beam.io import ReadFromText, WriteToText
from apache_beam.options.pipeline_options import PipelineOptions, SetupOptions

In [2]:
input_file = 'gs://dataflow-samples/shakespeare/kinglear.txt'
output_file = 'chapter2/output/output.txt'

In [3]:
# PipelineOptionsのインスタンス化
pipeline_options = PipelineOptions()

with beam.Pipeline(options=pipeline_options) as p:
    # テキストファイルをPCollectionに読み込み
    lines = p | ReadFromText(input_file)
    
    # 各単語の頻度を数える
    counts = (
        lines
        | 'Split' >> (beam.FlatMap(lambda x: re.findall(r'[A-Za-z\']+', x)))
        | 'PairWithOne' >> beam.Map(lambda x: (x, 1))
        | 'GroupAndSum' >> beam.CombinePerKey(sum)
    )
    
    # 頻度を文字列のPCollectionにフォーマット変換
    def format_result(word_count):
        (word, count) = word_count
        return "{}: {}".format(word, count)
    
    output = counts | 'Format' >> beam.Map(format_result)
    output | WriteToText(output_file)

Connecting anonymously.
